# Analyze results of simulations

In [ ]:
library(tidyverse)
library(rasilabRtemplates)
library(viridis)

In [ ]:
rxn_data <- list.files("output/", pattern = "rxn_list.tsv.gz", full.names = T) %>% 
  enframe() %>% 
  mutate(sim_id = as.integer(str_extract(value, "(?<=model_)\\d+"))) %>% 
  mutate(data = map(value, read_tsv)) %>%
  select(-name, -value) %>% 
  unnest() %>%
  print()

In [ ]:
sim_data <- list.files("output/", pattern = "molecule_type_list.tsv.gz", full.names = T) %>% 
  enframe() %>% 
  mutate(sim_id = as.integer(str_extract(value, "(?<=model_)\\d+"))) %>% 
  mutate(data = map(value, read_tsv)) %>%
  select(-name, -value) %>% 
  unnest() %>% 
  filter(mol_type %in% c("simulated_time", "cpu_time")) %>% 
  spread(mol_type, mol_type_id) %>% 
  print()

In [ ]:
last_rxn_data <- list.files("output/", pattern = "molecule_type_list.tsv.gz", full.names = T) %>% 
  enframe() %>% 
  mutate(sim_id = as.integer(str_extract(value, "(?<=model_)\\d+"))) %>% 
  mutate(data = map(value, read_tsv)) %>%
  select(-name, -value) %>% 
  unnest() %>% 
  filter(mol_type == "last_rxn_firing_time") %>%
  spread(mol_type, mol_type_id) %>% 
  print()

In [ ]:
check_rxn <- last_rxn_data %>%
  left_join(sim_data, by = "sim_id") %>%
  mutate(ratio = last_rxn_firing_time / simulated_time) %>%
  filter(ratio < 0.999) %>%
  print()

In [ ]:
annotations  <- read_tsv('sim.params.tsv') %>%
  print()

In [ ]:
psr_data <- rxn_data %>% 
  mutate(mrna_loc = str_extract(name, "\\d+$")) %>% 
  left_join(annotations %>% select(sim_id, uorf2_stop, orf_stop), by = "sim_id") %>% 
  filter((mrna_loc == orf_stop) & str_detect(name, "terminate")) %>% 
  group_by(sim_id, mrna_loc) %>% 
  dplyr::summarize(n_firings = sum(n_firings)) %>% 
  left_join(sim_data, by = "sim_id") %>% 
  mutate(psr = n_firings / simulated_time) %>% 
  select(sim_id, mrna_loc, psr) %>% 
  print()

In [ ]:
param_data <- annotations %>% 
    select(sim_id, k_cap_bind, k_elong_stall, k_start_uorf2, k_scan_term_3_hit_80s, k_terminated_ssu_recycle_uorf2) %>%
  print()

In [ ]:
data <- param_data %>% 
  left_join(psr_data, by = "sim_id") %>%
  dplyr::rename(k_recycle = k_terminated_ssu_recycle_uorf2,
         k_abort = k_scan_term_3_hit_80s
         ) %>% 
  mutate(k_recycle = round(k_recycle)) %>% 
  print()

In [ ]:
plot_data <- data %>%
  print()

plot_data %>% 
  ggplot(aes(x = log2(k_cap_bind), y = log2(psr), color = as.factor(k_elong_stall),  group = k_elong_stall)) +
  facet_wrap(~ k_abort + k_start_uorf2 + k_recycle, ncol = 3, labeller = label_both, scales = "free") +
  geom_point() +
  geom_line() +
  scale_color_manual(values = cbPalette[c(2,1)])

In [ ]:
expt_data <- read_csv("tables/nluc_fluc_control_mutants.csv") %>% 
  mutate(label = c("wt", "no_stall", "no_aug", "strong")) %>% 
  select(label, mean_ratio) %>%
  deframe() %>%
  print()

fit_data <- data %>% 
  group_by(across(starts_with("k_"))) %>% 
  ungroup(k_cap_bind) %>% 
  nest() %>% 
  mutate(fit = map(data, function(df) spline(x = df$k_cap_bind, y = df$psr, n = 100) %>% as.data.frame)) %>% 
  select(-data) %>%
  unnest(fit) %>%
  rename(k_cap_bind = x, psr = y) %>%
  print()

fit_data %>% 
  ungroup() %>% 
  # convert stall to be a binary factor
  mutate(stall = if_else(k_elong_stall == 0.001, T, F)) %>% 
  # convert uORF2 start strength to be binary
  mutate(kozak = if_else(k_start_uorf2 == 0.5, "WT", if_else(k_start_uorf2 == 0, "no_aug", "strong"))) %>% 
  # assign mutant to the same names as experiments for easy comparison
  mutate(mutant = case_when(
    stall == T & kozak == "WT" ~ 'wt',
    stall == T & kozak == "strong" ~ 'strong',
    stall == F & kozak == "WT" ~ 'no_stall',
    stall == T & kozak == "no_aug" ~ "no_aug")) %>% 
  # get rid of the of mutants that we are not minimizing (eg. strong kozak, no stall)
  filter(!is.na(mutant)) %>% 
  select(-stall, -kozak, -k_elong_stall, -k_start_uorf2) %>% 
  pivot_wider(names_from = "mutant", values_from = "psr") %>%
  # calculate the log2 difference between no_stall and wt / strong
  mutate(wt = log2(wt) - log2(no_aug)) %>% 
  mutate(strong = log2(strong) - log2(no_aug)) %>%
  mutate(no_stall = log2(no_stall) - log2(no_aug)) %>%
  # calculate RMS error
  mutate(rms = (wt - expt_data['wt'])^2 + (strong - expt_data['strong'])^2) %>% 
  # minimize RMS error in each model and for other parameters as needed
  group_by(k_abort) %>%
  arrange(rms) %>%
  slice(1) %>%
  write_csv("./tables/fit_k_cap_bind.csv") %>%
  knitr::kable()

other_data <- read_csv("../0c_andreev_vary_params_for_fitting_controls/tables/fit_params.csv") %>%
  mutate(Model = "80S-hit") %>%
  print()

plot_data_controls <- read_csv("./tables/fit_k_cap_bind.csv") %>%
  mutate(Model = if_else(k_abort == 0, "traffic jam-mediated enhanced repression", "collide and dissociate")) %>%
  full_join(other_data) %>%
  select(-rms, -k_recycle) %>%
  pivot_longer(cols = no_aug:strong, names_to = "mutant", values_to = "psr") %>%
  print()

plot_data_controls$mutant <- factor(plot_data_controls$mutant, levels=c("wt", "no_stall", "no_aug", "strong"), labels=c("wild-type", "no stall", "no start codon", "stronger Kozak"))

plot_data_controls %>%
  ggplot(aes(x = psr, y = mutant, shape = Model)) +
  geom_jitter() +
  scale_color_viridis(discrete = T, end = 0.9) +
  scale_x_continuous(breaks = scales::pretty_breaks(n=3)) +
  theme(axis.text.y = element_text(size = 7), axis.text.x = element_text(size = 7), axis.title.y=element_blank()) +
  labs(x = "log2 main ORF protein output (s-1)", y = "Mutant")
  
  ggsave('figures/fit_params_predicted_controls.pdf')